In [1]:
import Pkg
Pkg.activate("joint_timing")
Pkg.instantiate()

using Cuba, Distributions
using BenchmarkTools, Test, CUDA
using FLoops, FoldsCUDA
using SpecialFunctions

  Activating environment at `~/SageMaker/networks_hub/SAN/code/network_simulations/joint_timing/Project.toml`


In [15]:
M=25 # number of independent beta random variables
atol=1e-6
rtol=1e-3

# integrate the pdf of the joint distribution -- should always equal 1
function int(x, f)
   f[1] = pdf(Product(Beta.(1.0,2.0*ones(M))),x)
end

# multithread
function int_thread_col(x, f)
    Threads.@threads for i in 1:size(x,2)
      f[i] = pdf(Product(Beta.(1.0,2.0*ones(M))),@view(x[:,i]))
    end
end

# multithread and loop to create product distribution
function int_thread_el(x,f)
   f[1,:] .= 1.0
   Threads.@threads for j in 1:size(x,2)
       for i in 1:size(x, 1)
           f[1, j] *= pdf(Beta(1.0,2.0),@view(x[i,j]))
       end
   end
end

int_thread_el (generic function with 1 method)

In [23]:
function pdf_beta(x::Array,alpha::T,beta::T) where T
    x.^(alpha .- 1.0).*(1.0 .- x).^(beta .- 1.0)./SpecialFunctions.beta(1.0,2.0)
end

function pdf_beta(f,x,alpha,beta)
    f=x.^(alpha-1.0).*(1.0 .- x).^(beta-1.0)./SpecialFunctions.beta(1.0,2.0)
end

display(pdf_beta(0.2,1.0,2.0))
display(pdf_beta(0.0,0.2,1.0,2.0))

display(pdf_beta([0.2,0.3],1.0,2.0))
display(pdf_beta([0.0,0.0],[0.2,0.3],1.0,2.0))


1.6

1.6

2-element Vector{Float64}:
 1.6
 1.4

2-element Vector{Float64}:
 1.6
 1.4

In [29]:
function pdf_beta(x,a,b,dim)
    prod(x.^(a-1.0f0) .* (1.0f0 .- x).^(b-1.0f0)./(gamma(a)*gamma(b)/gamma(a+b)),dims=dim)
end


In [35]:
pdf(Product(Beta.(1.0,2.0*ones(2))),[0.2,0.3])


2.2399999999999998

In [30]:

nvec=15000000
maxevals=300000000
nmin=2
nnew=80000
flatness=150

@show result, err = suave(int, M, 1, atol=atol, rtol=rtol,maxevals=maxevals,nnew=nnew,nmin=nmin,flatness=flatness); 
@show result, err = suave(int2, M, 1, atol=atol, rtol=rtol,maxevals=maxevals,nnew=nnew,nmin=nmin,flatness=flatness); 


(result, err) = suave(int, M, 1, atol = atol, rtol = rtol, maxevals = maxevals, nnew = nnew, nmin = nmin, flatness = flatness) = Component:
 1: 0.9964268414111838 ± 0.0009925145844409677 (prob.: 1.0)
Integrand evaluations: 5360000
Number of subregions:  67
Note: The desired accuracy was reached


LoadError: MethodError: [0mCannot `convert` an object of type [92mVector{Float64}[39m[0m to an object of type [91mFloat64[39m
[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::Union{InitialValues.SpecificInitialValue{typeof(*)}, InitialValues.SpecificInitialValue{typeof(Base.mul_prod)}}[39m) where T<:Union{AbstractString, Number} at /home/ec2-user/.julia/packages/InitialValues/EPz1F/src/InitialValues.jl:257
[0m  convert(::Type{T}, [91m::LLVM.GenericValue[39m, [91m::LLVM.LLVMType[39m) where T<:AbstractFloat at /home/ec2-user/.julia/packages/LLVM/7Q46C/src/execution.jl:39
[0m  convert(::Type{T}, [91m::LLVM.ConstantFP[39m) where T<:AbstractFloat at /home/ec2-user/.julia/packages/LLVM/7Q46C/src/core/value/constant.jl:98
[0m  ...

In [2]:
# multithread
function int_thread_col(x, f)
    Threads.@threads for i in 1:size(x,2)
      f[i] = pdf(Product(Beta.(1.0,2.0*ones(M))),@view(x[:,i]))
    end
end

# multithread and loop to create product distribution
function int_thread_el(x,f)
   f[1,:] .= 1.0
   Threads.@threads for j in 1:size(x,2)
       for i in 1:size(x, 1)
           f[1, j] *= pdf(Beta(1.0,2.0),@view(x[i,j]))
       end
   end
end

In [2]:
# monte carlo suave

nvec=15000000
maxevals=300000000
nmin=2
nnew=80000
flatness=150

@show result, err = suave(int, M, 1, atol=atol, rtol=rtol,maxevals=maxevals,nnew=nnew,nmin=nmin,flatness=flatness); 
@show result, err = suave(int_thread_col, M, 1, atol=atol, rtol=rtol,maxevals=maxevals, nvec=nvec,nnew=nnew,nmin=nmin,flatness=flatness); 
@show result, err = suave(int_thread_el, M, 1, atol=atol, rtol=rtol,maxevals=maxevals, nvec=nvec,nnew=nnew,nmin=nmin,flatness=flatness);

In [2]:
# User Inputs
M= 5 # number of independent uniform random variables
atol=1e-10
rtol=1e-10
nvec=100000
maxevals=10000

# Initializing Matrices
ones_mat = CuArray(ones(Float32, M))
result = CUDA.ones(Float32, (nvec,1))
x_cpu = rand(Float64, (M, nvec))
x = CuArray(x_cpu)
x_1d_cpu = rand(Float64, M)
x_1d_gpu = CuArray(x_1d_cpu)

┌ Warning: Your Tesla K80 GPU does not meet the minimal required compute capability (3.7.0 < 5.0).
│ Some functionality might not work. For a fully-supported set-up, please use an older version of CUDA.jl
└ @ CUDA /home/ec2-user/.julia/packages/CUDA/k52QH/src/state.jl:224


5-element CuArray{Float64, 1}:
 0.09862817519574185
 0.37211988789676176
 0.9038049126217622
 0.2645842797714655
 0.934906268138505

In [3]:
@btime suave($(int), $M, 1, atol=$atol, rtol=$rtol,maxevals=$maxevals, nnew=$nnew,nmin=$nmin,flatness=$flatness) # fast

CuArray{Float32, 1}

In [7]:
typeof( CuArray(x_cpu))

CuArray{Float64, 2}

In [2]:
Pkg.test("CUDA")

     Testing CUDA
      Status `/tmp/jl_tUYAd6/Project.toml`
  [79e6a3ab] Adapt v3.3.0
  [ab4f0b2a] BFloat16s v0.1.0
  [052768ef] CUDA v3.1.0
  [864edb3b] DataStructures v0.18.9
  [7a1cc6ca] FFTW v1.4.1
  [1a297f60] FillArrays v0.11.7
  [0c68f7d7] GPUArrays v6.2.2
  [a98d9a8b] Interpolations v0.13.2
  [872c559c] NNlib v0.7.19
  [ade2ca70] Dates `@stdlib/Dates`
  [8ba89e20] Distributed `@stdlib/Distributed`
  [37e2e46d] LinearAlgebra `@stdlib/LinearAlgebra`
  [de0858da] Printf `@stdlib/Printf`
  [3fa0cd96] REPL `@stdlib/REPL`
  [9a3f8284] Random `@stdlib/Random`
  [2f01184e] SparseArrays `@stdlib/SparseArrays`
  [10745b16] Statistics `@stdlib/Statistics`
  [8dfed614] Test `@stdlib/Test`
      Status `/tmp/jl_tUYAd6/Manifest.toml`
  [621f4979] AbstractFFTs v1.0.1
  [79e6a3ab] Adapt v3.3.0
  [13072b0f] AxisAlgorithms v1.0.0
  [ab4f0b2a] BFloat16s v0.1.0
  [fa961155] CEnum v0.4.1
  [052768ef] CUDA v3.1.0
  [d360d2e6] ChainRulesCore v0.9.40
  [34da2185] Compat v3.27.0
  [864edb3b] DataStruc

                                          |          | ---------------- GPU ---------------- | ---------------- CPU ---------------- |
Test                             (Worker) | Time (s) | GC (s) | GC % | Alloc (MB) | RSS (MB) | GC (s) | GC % | Alloc (MB) | RSS (MB) |


┌ Warning: Your Tesla K80 GPU does not meet the minimal required compute capability (3.7.0 < 5.0).
│ Some functionality might not work. For a fully-supported set-up, please use an older version of CUDA.jl
└ @ CUDA ~/.julia/packages/CUDA/k52QH/src/state.jl:224
┌ Warning: Your Tesla K80 GPU does not meet the minimal required compute capability (3.7.0 < 5.0).
│ Some functionality might not work. For a fully-supported set-up, please use an older version of CUDA.jl
└ @ CUDA ~/.julia/packages/CUDA/k52QH/src/state.jl:224
┌ Warning: Your Tesla K80 GPU does not meet the minimal required compute capability (3.7.0 < 5.0).
│ Some functionality might not work. For a fully-supported set-up, please use an older version of CUDA.jl
└ @ CUDA ~/.julia/packages/CUDA/k52QH/src/state.jl:224
┌ Warning: Your Tesla K80 GPU does not meet the minimal required compute capability (3.7.0 < 5.0).
│ Some functionality might not work. For a fully-supported set-up, please use an older version of CUDA.jl
└ @ CUDA ~/.jul

apiutils                              (3) |     2.31 |   0.00 |  0.0 |       0.00 |    56.25 |   0.00 |  0.0 |       5.53 |   838.73 |
nvtx                                 (16) |     3.08 |   0.00 |  0.0 |       0.00 |    56.25 |   0.09 |  2.9 |      31.42 |   838.73 |
nvml                                 (15) |     6.13 |   0.00 |  0.0 |       0.00 |    56.25 |   0.11 |  1.8 |      50.11 |   838.73 |
curand                                (9) |     5.89 |   0.00 |  0.0 |       0.00 |    60.25 |   0.18 |  3.1 |      48.56 |   838.73 |
pointer                              (17) |    13.52 |   6.10 | 45.1 |       0.00 |    57.25 |   0.27 |  2.0 |     136.23 |   838.73 |
initialization                        (2) |    53.66 |   0.00 |  0.0 |       0.00 |    56.25 |   0.89 |  1.7 |     429.70 |   838.73 |
      From worker 11:	┌ Warning: Your Tesla K80 GPU does not meet the minimal required compute capability (3.7.0 < 5.0).
      From worker 11:	│ Some functionality might not work. For a full

┌ Warning: CUDA.CUDNN.cudnnDropoutSeed[] >= 0: dropout operations will be deterministic but 40x more expensive
└ @ CUDA.CUDNN ~/.julia/packages/CUDA/k52QH/lib/cudnn/dropout.jl:40


...

gpuarrays/math                        (5) |    53.04 |   0.02 |  0.0 |       0.00 |    61.75 |   1.78 |  3.4 |     695.13 |   870.58 |
cusolver/sparse                      (14) |   215.37 |   0.01 |  0.0 |       0.19 |   202.75 |   6.25 |  2.9 |    2323.41 |  1390.13 |
      From worker 11:	┌ Warning: Your Tesla K80 GPU does not meet the minimal required compute capability (3.7.0 < 5.0).
      From worker 11:	│ Some functionality might not work. For a fully-supported set-up, please use an older version of CUDA.jl
      From worker 11:	└ @ CUDA ~/.julia/packages/CUDA/k52QH/src/state.jl:224
gpuarrays/input output               (14) |    29.56 |   0.02 |  0.1 |       0.00 |    57.25 |   2.45 |  8.3 |     249.41 |  1390.13 |
cusolver/multigpu                    (18) |   329.27 |   0.22 |  0.1 |     431.84 |  1356.75 |   9.82 |  3.0 |    3569.91 |  1068.85 |
device/ldg                            (8) |   150.86 |   0.02 |  0.0 |       0.00 |    61.75 |   4.07 |  2.7 |    1874.04 |  1048.38 |

┌ Warning: Your Tesla K80 GPU does not meet the minimal required compute capability (3.7.0 < 5.0).
│ Some functionality might not work. For a fully-supported set-up, please use an older version of CUDA.jl
└ @ CUDA ~/.julia/packages/CUDA/k52QH/src/state.jl:224


examples                             (11) |  1088.26 |   0.00 |  0.0 |       0.00 |    56.25 |   0.09 |  0.0 |      44.51 |   838.73 |
gpuarrays/random                      (8) |   300.52 |   0.03 |  0.0 |       0.03 |    63.75 |   8.42 |  2.8 |    3382.40 |  1052.46 |
execution                            (13) |  1131.12 |   5.81 |  0.5 |       0.02 |   109.62 |  31.81 |  2.8 |   12604.10 |  1003.78 |
gpuarrays/base                        (6) |   315.41 |   0.01 |  0.0 |      17.44 |   101.75 |  13.27 |  4.2 |    4701.24 |  1510.96 |
cublas                                (7) |  1197.80 |   5.79 |  0.5 |      14.49 |   129.75 |  33.81 |  2.8 |   14570.33 |  1152.14 |
array                                 (4) |  1219.01 |   6.64 |  0.5 |       5.29 |    82.88 |  41.76 |  3.4 |   14831.12 |  1005.93 |
cusolver/dense                       (16) |   886.27 |   0.67 |  0.1 |    1294.29 |   242.75 |  29.91 |  3.4 |   12983.83 |  1535.34 |
sorting                              (15) |  1390.19 | 

ERROR: LoadError: Test run finished with errors
in expression starting at /home/ec2-user/.julia/packages/CUDA/k52QH/test/runtests.jl:490


LoadError: Package CUDA errored during testing